In [32]:
import pandas as pd
# from cscore.utils import  transform_files
import glob
import numpy as np
import pandas as pd
import glob
# import win32com.client as win32
import jaydebeapi
import re
import sys
from datetime import datetime
from dotenv import load_dotenv
load_dotenv() 
import os

sys.path.append(os.path.abspath(''))
from utils import clean_column_names,all_history,transform_files


In [31]:
read_file('./cscore/input/acc_current/',file_current)

,BaseDate,Loan No,Product Code,Overdue Start Date,OverdueDays_Morning,OverdueCnt_Morning,OverdueDays,OverdueCnt,Loan Amt,Loan Balance,...,OverdueCrUsage,OverdueOutputTax,OverdueCollectionFee,OverdueVAT,OverdueStampTaxAmt,OverdueNCBTaxAmt,OverdueETCFeeAmt,LawCost,LastPaymentDt,LastPaymentAmt
0,20250401,9010221122000949,A501:TL,2024-11-01,151,5:OD4,151,5:OD4,74164.44,33528.80,...,792.37,0.0,467.3,32.7,0,0,0.0,0.0,2025-03-31,3067.00
1,20250401,9010221122000931,A501:TL,,0,99:NORMAL,0,99:NORMAL,41659.76,11088.74,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2024-09-27,20971.92
2,20250401,9010221122000922,A501:TL,2023-08-31,0,98:Write-Off,0,98:Write-Off,34238.99,23851.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2023-01-31,1300.00
3,20250401,9010221122000914,A501:TL,2024-02-29,0,98:Write-Off,0,98:Write-Off,16118.33,13559.55,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2023-12-27,550.00
4,20250401,9010221122000906,A501:TL,2022-11-30,0,98:Write-Off,0,98:Write-Off,35235.40,31005.17,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2022-09-30,2100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398867,20250401,1000158090014831,A401:ML,2017-02-01,0,99:NORMAL,0,99:NORMAL,25500.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-01-31,1580.00
1398868,20250401,1000158090014814,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,24600.00,30.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-07-31,1260.00
1398869,20250401,1000158090008580,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-07-16,4000.00
1398870,20250401,1000158090005394,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-08-01,2600.00


In [12]:
conn = jaydebeapi.connect(
    os.getenv("ORACLE_JCLASSNAME"),
        os.getenv("ORACLE_URL"),
        [os.getenv("ORACLE_USER"), os.getenv("ORACLE_PASSWORD")],
        os.getenv("ORACLE_JARS"),
    )

cur = conn.cursor() 	

query = pd.read_sql(f'''
    SELECT * FROM supat.bucket_score  
    ''', conn) 
print('query',query)
data_performance = pd.read_sql(f'''
    WITH contract AS (
    SELECT AS_OF_DATE ,CONTRACT_NO ,NATIONAL_ID ,CONTRACT_DATE ,PRINCIPAL_BAL 
    FROM JFDWH.CONTRACT_INFO_DAILY 
    WHERE AS_OF_DATE = (SELECT MAX(AS_OF_DATE) FROM JFDWH.CONTRACT_INFO_DAILY)-1
    OFFSET 5 ROWS FETCH NEXT 100 ROWS ONLY
    )
    ,
    rawdata AS (
    SELECT c.*,c2.NAME ,c2.SURNAME ,c2.GROSS_INCOME ,c2.DATE_OF_BIRTH ,c2.BUSINESS_TYPE ,c2.OCCUPATION_TYPE,ro.OCCUPATION_NAME  FROM contract c
    LEFT JOIN JFDWH.CUSTOMER c2 ON c2.NATIONAL_ID_NO = c.NATIONAL_ID
    LEFT JOIN REF_OCCUPATION ro ON c2.OCCUPATION_TYPE = ro.ID 
    )
    ,
    data AS (
    SELECT tcc.AS_OF_DATE,tcc.CONTRACT_NO ,tcc.NATIONAL_ID ,tcc.GROSS_INCOME ,
    tcc.OCCUPATION_TYPE ,tcc.OCCUPATION_NAME,
    tcc.CONTRACT_DATE ,floor(MONTHS_BETWEEN(SYSDATE, tcc.CONTRACT_DATE))  AS contract_period,
    FLOOR(MONTHS_BETWEEN(SYSDATE, tcc.DATE_OF_BIRTH) / 12) AS age,
    PRINCIPAL_BAL ,
    CASE WHEN floor(MONTHS_BETWEEN(SYSDATE, tcc.CONTRACT_DATE))  < 7 THEN 6 ELSE 7 END AS MOB,
    CAST(COALESCE(tis.INDEX_SCORE , '1') AS NUMBER) AS digit2income,
        CAST(COALESCE(tas.INDEX_SCORE, '3') AS NUMBER) AS digit2age,
        CAST(COALESCE(tos.INDEX_SCORE, '3') AS NUMBER) AS digit2job
    FROM rawdata tcc 
    LEFT JOIN TEMP_AGE_SCORE tas ON tas.AGE = FLOOR(MONTHS_BETWEEN(SYSDATE, tcc.DATE_OF_BIRTH) / 12) 
    LEFT JOIN TEMP_OCCUPATION_SCORE tos ON tos.OCCUPATION_TYPE  = tcc.OCCUPATION_TYPE
    LEFT JOIN Temp_INCOME_SCORE tis ON tis.INCOME = tcc.GROSS_INCOME
    )
    SELECT t2.*,
    t3.INDEX_SCORE AS TOTAL_DIGIT2
    FROM  data t2
    LEFT JOIN TEMP_DIGIT2_SCORE t3 ON t2.digit2income + t2.digit2age + t2.digit2job = t3.SUMDIGIT  
    ''', conn) 
print(data_performance)
conn.close()
# =======================ตั้งต้น===================================

file_current = glob.glob("./cscore/input/acc_current/*") 
file_current = file_current[0].split('\\')[1]
data_digit1 = read_file('./cscore/input/acc_current/',file_current)
data_digit1 = clean_column_names(data_digit1)
# data_digit1 = pd.read_csv(f'./input/acc_current/{file_current}')
data_digit1['contract_no'] = data_digit1['contract_no'].astype(str)
data_digit1 = data_digit1[['contract_no']]


files_history = glob.glob("./cscore/input/acc_history_monthly/*.csv") 
# print(f'file_history :, {files_history} ?? ')


processed_files = []  # Store modified file names
for i in files_history:
    # print()
    file = i.split('\\')[-1] 
    # print(file)
    processed_files.append(file)  
his = transform_files(processed_files)

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_3632\769264389.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query = pd.read_sql(f'''
C:\Users\Sireetron\AppData\Local\Temp\ipykernel_3632\769264389.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_performance = pd.read_sql(f'''


query     ID        BUCKET SCORE_INDEX
0    1     99:NORMAL           1
1    2          1:XD           1
2    3         2:OD1           2
3    4         3:OD2           2
4    5         4:OD3           3
5    6         5:OD4           3
6    7         6:OD5           3
7    8         7:OD6           3
8    9         8:OD7           3
9   10         9:OD8           3
10  11           10:           3
11  12           11:           3
12  13       12:OVER           3
13  14  98:WRITE-OFF           3
             AS_OF_DATE       CONTRACT_NO    NATIONAL_ID  GROSS_INCOME  \
0   2025-04-18 00:00:00  1000158080013009  3110100189044          8000   
1   2025-04-18 00:00:00  1000158080008323  1309900376477         11146   
2   2025-04-18 00:00:00  1000158080018370  1309900788911         13820   
3   2025-04-18 00:00:00  1000158080018655  1709900339306         15372   
4   2025-04-18 00:00:00  1000158080003186  5589990000189         15500   
..                  ...               ...            ..

In [13]:
query

,ID,BUCKET,SCORE_INDEX
0,1,99:NORMAL,1
1,2,1:XD,1
2,3,2:OD1,2
3,4,3:OD2,2
4,5,4:OD3,3
5,6,5:OD4,3
6,7,6:OD5,3
7,8,7:OD6,3
8,9,8:OD7,3
9,10,9:OD8,3


In [ ]:
all_history('./cscore/input/acc_history_monthly/',query,i[0], i[1])

In [38]:
def all_history(path,query,date,item) :
    print('start')
    data_mnt = pd.read_csv(f'{path}{date}')
    print(f'./input/history/{date}')
    # print(data_mnt)
    data_mnt['Loan No'] = data_mnt['Loan No'].astype(str)
    data_mnt = data_mnt[['Loan No','OverdueCnt_Morning']].merge(query, left_on='OverdueCnt_Morning', right_on='BUCKET', how='left')
    print(data_mnt)
    data_mnt = data_mnt.rename(columns={'BUCKET_SCORE': f'BUCKET_SCORE{item}mnt'})
    
    return data_mnt[['Loan No',f'BUCKET_SCORE{item}mnt']]

In [39]:
for i in his[0:]:
    # print(f"Processing date: {i[0]}")
    # print((i[0], i[1]))
    data = all_history('./cscore/input/acc_history_monthly/',query,i[0], i[1])
    data = clean_column_names(data)
    data_digit1 = data_digit1.merge(data, on='contract_no', how='left')

start
./input/history/Acc_20250401_0830.csv
                  Loan No OverdueCnt_Morning   ID     BUCKET SCORE_INDEX
0        9010221122000949              5:OD4  6.0      5:OD4           3
1        9010221122000931          99:NORMAL  1.0  99:NORMAL           1
2        9010221122000922       98:Write-Off  NaN        NaN         NaN
3        9010221122000914       98:Write-Off  NaN        NaN         NaN
4        9010221122000906       98:Write-Off  NaN        NaN         NaN
...                   ...                ...  ...        ...         ...
1398867  1000158090014831          99:NORMAL  1.0  99:NORMAL           1
1398868  1000158090014814          99:NORMAL  1.0  99:NORMAL           1
1398869  1000158090008580          99:NORMAL  1.0  99:NORMAL           1
1398870  1000158090005394          99:NORMAL  1.0  99:NORMAL           1
1398871  1000158080014811          99:NORMAL  1.0  99:NORMAL           1

[1398872 rows x 5 columns]


KeyError: "['BUCKET_SCOREAcc_20250401_0830mnt'] not in index"